In [30]:
#Import dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import time

####SCRAPING CODE#######
##Scraping Google News Aggregator for most recent transit news stories###
# URL of page to be scraped
transit_news = 'https://news.google.com/topics/CAAqIQgKIhtDQkFTRGdvSUwyMHZNRGRpYzNrU0FtVnVLQUFQAQ/sections/CAQqEAgAKgcICjCj9vAKMM_ruwIwqaLKBQ?hl=en-US&gl=US&ceid=US%3Aen'
# Retrieve page with the requests module
news_response = requests.get(transit_news)
# Create BeautifulSoup object; parse with 'lxml'
news_soup = bs(news_response.text, 'html')
#Find first headline to collect latest news title and paragraph text.
headlines = news_soup.find_all('article')


In [33]:
#Extend headlines result to find h3s that have 
result = []
for h in headlines:
    result.extend(h.find_all('h3'))

In [36]:
result

[<h3 class="ipQwMb ekueJc RD0gLb"><a class="DY5T1d" href="./articles/CAIiEDMg9mo_lsJXe1D5VA51ogMqGAgEKg8IACoHCAowpbDpAzCm_hwwgvDGBg?hl=en-US&amp;gl=US&amp;ceid=US%3Aen">Pandemic has surprising impacts on public transit demand</a></h3>,
 <h3 class="ipQwMb ekueJc RD0gLb"><a class="DY5T1d" href="./articles/CAIiEJspU3LqDhI91BVtkgkGp2EqGQgEKhAIACoHCAow-uGXCzChi68DMNzFzwY?hl=en-US&amp;gl=US&amp;ceid=US%3Aen">Eastern Shore company to pay $550K for misleading Boston-area public transit agency over hand sanitizer</a></h3>,
 <h3 class="ipQwMb ekueJc RD0gLb"><a class="DY5T1d" href="./articles/CAIiEFncySqqJUFpnxCl4R00TJYqFwgEKg8IACoHCAow-4fWBzD4z0gwwtp6?hl=en-US&amp;gl=US&amp;ceid=US%3Aen">New York and the crisis in mass transit systems</a></h3>,
 <h3 class="ipQwMb ekueJc RD0gLb"><a class="DY5T1d" href="./articles/CAIiEBtZNz2VnJzgDUDKFfHlJeEqMwgEKioIACIQ2ZVU06V7rhBlms0-gSI93SoUCAoiENmVVNOle64QZZrNPoEiPd0w07jpBg?hl=en-US&amp;gl=US&amp;ceid=US%3Aen">Mass Transit hopes to get donated buses rolling ne

In [49]:
#Append headline text to its own list
headline_text = []
for i in range(0,70):
    headline_text.append(result[i].text)

In [52]:
headline_text[1]

'Eastern Shore company to pay $550K for misleading Boston-area public transit agency over hand sanitizer'

In [53]:
#Combine text fxn to make new version of headline_text that's just
#One big string.
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text


In [54]:
#Make big string version of recent headlines 
headlines_string = combine_text(headline_text)
headlines_string

'Pandemic has surprising impacts on public transit demand Eastern Shore company to pay $550K for misleading Boston-area public transit agency over hand sanitizer New York and the crisis in mass transit systems Mass Transit hopes to get donated buses rolling next month Google Maps to share COVID-19 case count and public transport occupancy details Zurich to use rideshare service to plug public transport gaps Rapid transit arrives in Omaha with launch of ORBT Mass Transit seeks approval for bus donation Central Subway opening pushed to 2022 Collier\'s 10-year public transit plan details host of recommended improvements Call-a-Ride, Delta Public Transit receive State CARES funds for safety protections Crisis Paves the Way for Transit Payment Innovation OSU study finds varied impact of pandemic on public transit use across US Lodz tests a device that kills almost all bacteria and viruses on public transport New public transit on the way for Idaho Falls Best Practices - Creating a Safer, Mo

In [58]:
#Preprocess text for analysis
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('@[^\s]+','',text)
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    return text

headlines_cleaned = clean_text_round1(headlines_string)

In [59]:
#Look at headlines cleaned to see if it makes sense 
headlines_cleaned

'pandemic has surprising impacts on public transit demand eastern shore company to pay  for misleading bostonarea public transit agency over hand sanitizer new york and the crisis in mass transit systems mass transit hopes to get donated buses rolling next month google maps to share  case count and public transport occupancy details zurich to use rideshare service to plug public transport gaps rapid transit arrives in omaha with launch of orbt mass transit seeks approval for bus donation central subway opening pushed to  colliers  public transit plan details host of recommended improvements callaride delta public transit receive state cares funds for safety protections crisis paves the way for transit payment innovation osu study finds varied impact of pandemic on public transit use across us lodz tests a device that kills almost all bacteria and viruses on public transport new public transit on the way for idaho falls best practices  creating a safer more secure ride on metro transit 

In [60]:
#Get NLTK tools in 
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flier\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\flier\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [61]:
#Tokenize big string
headlines_tokenized = word_tokenize(headlines_cleaned)

In [62]:
headlines_tokenized

['pandemic',
 'has',
 'surprising',
 'impacts',
 'on',
 'public',
 'transit',
 'demand',
 'eastern',
 'shore',
 'company',
 'to',
 'pay',
 'for',
 'misleading',
 'bostonarea',
 'public',
 'transit',
 'agency',
 'over',
 'hand',
 'sanitizer',
 'new',
 'york',
 'and',
 'the',
 'crisis',
 'in',
 'mass',
 'transit',
 'systems',
 'mass',
 'transit',
 'hopes',
 'to',
 'get',
 'donated',
 'buses',
 'rolling',
 'next',
 'month',
 'google',
 'maps',
 'to',
 'share',
 'case',
 'count',
 'and',
 'public',
 'transport',
 'occupancy',
 'details',
 'zurich',
 'to',
 'use',
 'rideshare',
 'service',
 'to',
 'plug',
 'public',
 'transport',
 'gaps',
 'rapid',
 'transit',
 'arrives',
 'in',
 'omaha',
 'with',
 'launch',
 'of',
 'orbt',
 'mass',
 'transit',
 'seeks',
 'approval',
 'for',
 'bus',
 'donation',
 'central',
 'subway',
 'opening',
 'pushed',
 'to',
 'colliers',
 'public',
 'transit',
 'plan',
 'details',
 'host',
 'of',
 'recommended',
 'improvements',
 'callaride',
 'delta',
 'public',
 'tr

In [63]:
#Take out stop words 
stop_words = set(stopwords.words('english')) 
headlines_nostopwords = [w for w in headlines_tokenized if not w in stop_words]
headlines_nostopwords

['pandemic',
 'surprising',
 'impacts',
 'public',
 'transit',
 'demand',
 'eastern',
 'shore',
 'company',
 'pay',
 'misleading',
 'bostonarea',
 'public',
 'transit',
 'agency',
 'hand',
 'sanitizer',
 'new',
 'york',
 'crisis',
 'mass',
 'transit',
 'systems',
 'mass',
 'transit',
 'hopes',
 'get',
 'donated',
 'buses',
 'rolling',
 'next',
 'month',
 'google',
 'maps',
 'share',
 'case',
 'count',
 'public',
 'transport',
 'occupancy',
 'details',
 'zurich',
 'use',
 'rideshare',
 'service',
 'plug',
 'public',
 'transport',
 'gaps',
 'rapid',
 'transit',
 'arrives',
 'omaha',
 'launch',
 'orbt',
 'mass',
 'transit',
 'seeks',
 'approval',
 'bus',
 'donation',
 'central',
 'subway',
 'opening',
 'pushed',
 'colliers',
 'public',
 'transit',
 'plan',
 'details',
 'host',
 'recommended',
 'improvements',
 'callaride',
 'delta',
 'public',
 'transit',
 'receive',
 'state',
 'cares',
 'funds',
 'safety',
 'protections',
 'crisis',
 'paves',
 'way',
 'transit',
 'payment',
 'innovation'

In [66]:
#Analyze polarity/subjectivity
from textblob import TextBlob

final_list = combine_text(headlines_nostopwords)
pol = TextBlob(final_list)
headline_polarity = pol.sentiment.polarity
headline_subjectivity = pol.sentiment.subjectivity

headline_subjectivity

0.2134411121524523

In [67]:
#Make little df
d = {'category': ['polarity', 'subjectivity'], 'value': [headline_polarity, headline_subjectivity]}
df = pd.DataFrame(data=d)
df

,category,value
0,polarity,0.086867
1,subjectivity,0.213441


In [68]:
df.to_csv('headline_nlp.csv')